# Aplicações práticas

# 2.1 Exercício 1: Aborto

## A. Enunciado 

Numa amostra aleatória de 156 respondentes, um responsável pelo ministério da saúde pretende analisar a associação entre a opinião sobre o aborto quando a saúde da mãe está em perigo, (smãe) e quando a mãe é pobre (pobre). Obtiveram-se 16 pessoas a favor de ambos os abortos, 76 pessoas contra os dois tipos de aborto, 58 a favor de pobre e contra mãe, 6 contra pobre e a favor de smãe. As variáveis são ambas nominais, pois definem-se apenas pelo nome. As categorias de ambas as variáveis são: 1. Favor e 2. Contra, originando uma tabela 2 x 2. Os dados foram obtidos com base num questionário lançado em setembro (estudo transversal) e constam da linha Count da Tabela 1 do ficheiro cap02_exercicio1.sav e organizados em um dicionário em Python.


# Resolução em Python

## Carregando Bibliotecas

In [1]:
import pandas as pd  # Fornece DataFrames e Series para manipulação de dados em Python, facilitando operações como leitura, escrita, e manipulação de estruturas de dados tabulares.
import numpy as np  # Suporta operações com arrays e matrizes, oferecendo uma vasta gama de funções matemáticas para operações com essas estruturas de dados.
from statsmodels.stats.contingency_tables import Table  # Analisa tabelas de contingência para estudo de variáveis categóricas, útil em testes de hipóteses e análises de associação entre variáveis.
from scipy.stats import hypergeom  # Aplica a distribuição hipergeométrica para análises estatísticas, útil em testes de sobreposição e em situações onde se deseja calcular probabilidades sem reposição.

## Funções Personalizadas

In [2]:
import sys

# Adicionar o caminho do diretório ao sys.path
sys.path.append(r"C:\Users\ricar\OneDrive\Área de Trabalho\Livro\Cap_02\socialdataanalysis")

from association import analisar_independencia_variaveis_tabela_contingencia
from association import gerar_tabela_contingencia
from association import calcular_distribuicao_probabilidades_e_decisao_hipotese
from association import complementar_tabela_contingencia_com_analise_estatistica
from association import calcular_odds_ratio_razao_risco_discrepancia

## Dados Fornecidos

In [3]:
# Dicionário com dados categorizados representando grupos e suas frequências
dados = {
    'Pobre': [1, 1, 2, 2],
    'Smãe': [1, 2, 1, 2],
    'Freq': [16, 58, 6, 76]
}

# Criando o DataFrame a partir do dicionário
dados = pd.DataFrame(dados)

# Convertendo as colunas para inteiros, se necessário
dados = dados.astype(int)

# Visualizando o dataframe
display(dados)

,Pobre,Smãe,Freq
0,1,1,16
1,1,2,58
2,2,1,6
3,2,2,76


## Tabela 1: Tabela de Contingência

In [4]:
# Obter automaticamente os nomes dos grupos e da coluna de frequência
grupos = dados.columns[:-1].tolist()  # Todos exceto a última coluna
categorias = {
    grupos[0]: ['1. Favor de Pobre', '2. Contra Pobre',],
    grupos[1]: ['1. Favor de Smãe', '2. Contra Smãe'],
}

tabela_contingencia = gerar_tabela_contingencia(dados, grupos, categorias)

display(tabela_contingencia)

Smãe,1. Favor de Smãe,2. Contra Smãe
Pobre,,
1. Favor de Pobre,16,58
2. Contra Pobre,6,76


## Tabela de Contingência com Cálculos

In [5]:
# Função própria complementar_tabela_contingencia_com_analise_estatistica
combined_df = complementar_tabela_contingencia_com_analise_estatistica(tabela_contingencia)

# Visualizando resultado
display(combined_df)

Smãe                                    1. Favor de Smãe 2. Contra Smãe  Total
Pobre                                                                         
1. Favor de Pobre Count                             16.0           58.0   74.0
                  Expected Count                    10.4           63.6   74.0
                  % within Pobre                    21.6           78.4  100.0
                  % within Smãe                     72.7           43.3   47.4
                  % of Total                        10.3           37.2   47.4
                  Standardized Residual              1.7           -0.7       
                  Adjusted Residual                  2.6           -2.6       
2. Contra Pobre   Count                              6.0           76.0   82.0
                  Expected Count                    11.6           70.4   82.0
                  % within Pobre                     7.3           92.7  100.0
                  % within Smãe                     27.3           56.7   52.6
                  % of Total                         3.8           48.7   52.6
                  Standardized Residual             -1.6            0.7       
                  Adjusted Residual                 -2.6            2.6       
Total             Count                             22.0          134.0  156.0
                  Expected Count                    22.0          134.0  156.0
                  % within Pobre                    14.1           85.9  100.0
                  % within Smãe                    100.0          100.0  100.0
                  % of Total                        14.1           85.9  100.0

<font color="blue">    
    
### 2.1.1. Probabilidades e Teorema de Bayes
</font>

<font color="blue">    
    
#### a) Probabilidade de ser a favor do aborto por pobreza (Favor de Pobre).
</font>

In [6]:
## PROBABILIDADES MARGINAIS ##

# Calculando as Probabilidades Marginais para o Grupo 1
prob_marg_grupo0 = tabela_contingencia.sum(axis=1) / tabela_contingencia.values.sum()
# Convertendo em um DataFrame
prob_marg_grupo0 = prob_marg_grupo0.to_frame(name='')

# Exibindo os DataFrames
print(f"Probabilidades Marginais de {grupos[0]}")
display(prob_marg_grupo0.round(3))

# Impressão de configuração específica
evento = categorias[grupos[0]][0]
print(f"RESPOSTA: \n P({evento}) = {prob_marg_grupo0.loc[evento, '']:.3f}")

Probabilidades Marginais de Pobre


,
Pobre,
1. Favor de Pobre,0.474
2. Contra Pobre,0.526


RESPOSTA: 
 P(1. Favor de Pobre) = 0.474


<font color="blue">
    
#### b) Probabilidade de ser a favor do aborto quando a saúde da mãe está em risco (Favor de Smãe).
</font>

In [7]:
## PROBABILIDADES MARGINAIS ##

# Calculando as Probabilidades Marginais para o Grupo 2
prob_marg_grupo1 = tabela_contingencia.sum(axis=0) / tabela_contingencia.values.sum()
# Convertendo em um DataFrame
prob_marg_grupo1 = prob_marg_grupo1.to_frame(name='').T

# Exibindo os DataFrames
print(f"Probabilidades Marginais de {grupos[1]}")
display(prob_marg_grupo1.round(3))

# Impressão de configuração específica
evento = categorias[grupos[1]][0]
print(f"RESPOSTA: \n P({evento}) = {prob_marg_grupo1.loc['', evento]:.3f}")

Probabilidades Marginais de Smãe


Smãe,1. Favor de Smãe,2. Contra Smãe
,0.141,0.859


RESPOSTA: 
 P(1. Favor de Smãe) = 0.141


<font color="blue">

#### c) Probabilidade da ocorrência favorável aos dois tipos de aborto.
</font>

In [8]:
## PROBABILIDADES CONJUNTAS ##

# Dividindo cada frequência pelo total geral para obter probabilidades conjuntas
prob_conjuntas = tabela_contingencia / tabela_contingencia.sum().sum()

# Exibindo as probabilidades conjuntas com arredondamento para 3 casas decimais
display(prob_conjuntas.round(3))

# Impressão de configuração específica
evento_grupo0 = categorias[grupos[0]][0]
evento_grupo1 = categorias[grupos[1]][0]
print(f"RESPOSTA: \n P({evento_grupo0} ^ {evento_grupo1}) = {prob_conjuntas.loc[evento_grupo0, evento_grupo1]:.3f}")

Smãe,1. Favor de Smãe,2. Contra Smãe
Pobre,,
1. Favor de Pobre,0.103,0.372
2. Contra Pobre,0.038,0.487


RESPOSTA: 
 P(1. Favor de Pobre ^ 1. Favor de Smãe) = 0.103


<font color="blue">

#### d) Probabilidade de ser a Favor de Smãe, sabendo que é a Favor de Pobre.
</font>

In [9]:
## PROBABILIDADES CONDICIONADAS ##

# Calculando P(grupo1|grupo0)
prob_cond_grupo1_dado_grupo0 = tabela_contingencia.div(tabela_contingencia.sum(axis=1), axis=0)

# Exibindo as probabilidades condicionais com arredondamento para 3 casas decimais
print(f"Probabilidade Condicionada P({grupos[1]}|{grupos[0]}):")
display(prob_cond_grupo1_dado_grupo0.round(3))

# Impressão de configuração específica
evento = categorias[grupos[1]][0]
condicao = categorias[grupos[0]][0]
print(f"RESPOSTA: \n P({evento}|{condicao}) = {prob_cond_grupo1_dado_grupo0.loc[condicao, evento]:.3f}")

Probabilidade Condicionada P(Smãe|Pobre):


Smãe,1. Favor de Smãe,2. Contra Smãe
Pobre,,
1. Favor de Pobre,0.216,0.784
2. Contra Pobre,0.073,0.927


RESPOSTA: 
 P(1. Favor de Smãe|1. Favor de Pobre) = 0.216


<font color="blue">
    
#### e)	Probabilidade de ser a Favor de Pobre, sabendo que é a Favor de Smãe. 
</font>

In [10]:
## PROBABILIDADES CONDICIONADAS ##

# Calculando P(grupo0|grupo1)
prob_cond_grupo0_dado_grupo1 = tabela_contingencia.div(tabela_contingencia.sum(axis=0), axis=1)

# Exibindo as probabilidades condicionadas com arredondamento para 3 casas decimais
print(f"\n Probabilidade Condicionada P({grupos[0]}|{grupos[1]}):")
display(prob_cond_grupo0_dado_grupo1.round(3))

# Impressão de configuração específica
evento = categorias[grupos[0]][0]
condicao = categorias[grupos[1]][0]
print(f"RESPOSTA: \n P({evento}|{condicao}) = {prob_cond_grupo0_dado_grupo1.loc[evento, condicao]:.3f}")


 Probabilidade Condicionada P(Pobre|Smãe):


Smãe,1. Favor de Smãe,2. Contra Smãe
Pobre,,
1. Favor de Pobre,0.727,0.433
2. Contra Pobre,0.273,0.567


RESPOSTA: 
 P(1. Favor de Pobre|1. Favor de Smãe) = 0.727


<font color="blue">

#### f)	Probabilidade de ser contra o aborto por pobreza, sabendo que é a Favor de Smãe.
</font>

In [11]:
## PROBABILIDADES CONDICIONADAS ##

evento = categorias[grupos[0]][1]
condicao = categorias[grupos[1]][0]
print(f"RESPOSTA: \n P({evento}|{condicao}) = {prob_cond_grupo0_dado_grupo1.loc[evento, condicao]:.3f}")

RESPOSTA: 
 P(2. Contra Pobre|1. Favor de Smãe) = 0.273


<font color="blue">    
    
### 2.1.2. Frequência esperada
</font>

<font color="blue">
    
#### g) A frequência que se espera obter nas opiniões favoráveis aos dois tipos de aborto.
</font>

In [12]:
## FREQUÊNCIAS ESPERADAS ##

# Encapsulando a tabela para análises no statsmodels
tabela_analise = Table(tabela_contingencia)

# Calculando as frequências esperadas (fe)
frequencias_esperadas = tabela_analise.fittedvalues

# Exibindo as fe
display(frequencias_esperadas.round(1))

# Impressão de configuração específica
evento_grupo0 = categorias[grupos[0]][0]
evento_grupo1 = categorias[grupos[1]][0]
print(f"RESPOSTA: \n Fe({evento_grupo0} e {evento_grupo1}) = {frequencias_esperadas.loc[evento_grupo0, evento_grupo1]:.1f}")

Smãe,1. Favor de Smãe,2. Contra Smãe
Pobre,,
1. Favor de Pobre,10.4,63.6
2. Contra Pobre,11.6,70.4


RESPOSTA: 
 Fe(1. Favor de Pobre e 1. Favor de Smãe) = 10.4


<font color="blue">    
    
### 2.1.3. Teste de independência do Qui-Quadrado de Pearson
</font>

<font color="blue">
    
#### h) A significância da associação, usando o teste de independência do Qui-Quadrado de Pearson.
</font>

In [13]:
analisar_independencia_variaveis_tabela_contingencia(tabela_contingencia, 
                                                     mostrar_pearson=True, 
                                                     mostrar_continuity=True, 
                                                     mostrar_likelihood=True, 
                                                     mostrar_fisher=True)

,Value,df,Asymp. Sig. (2-sided),Exact Sig. (2-sided),Exact Sig. (1-sided)
Test,,,,,
Pearson Chi-Square,6.57,1,0.010,,
Continuity Correction,5.44,1,0.020,,
Likelihood Ratio,6.73,1,0.009,,
Fisher`s Exact Test,3.49,,,0.012,0.009
N. of Valid Cases,156,,,,


a. 0 cells (0.00%) have expected count less than 5. The minimum expected count is 10.44.


<font color="blue">
    
### 2.1.4. Resíduos Ajustados Estandardizados
</font>

<font color="blue">

#### i)	A significância da associação, usando os resíduos ajustados estandardizados.
</font>

In [14]:
# Calcula os Resíduos de Pearson (não ajustados)
residuos_nao_ajustados = tabela_analise.resid_pearson

# Calcula os Resíduos ajustados estandardizados
residuos_estandardizados = tabela_analise.standardized_resids

# Visualizando
print(f"\nRESPOSTA: \n")
display(residuos_estandardizados.round(1))


RESPOSTA: 



Smãe,1. Favor de Smãe,2. Contra Smãe
Pobre,,
1. Favor de Pobre,2.6,-2.6
2. Contra Pobre,-2.6,2.6


<font color="blue">
    
### 2.1.5. Odds Ratio e Racio de Prevalência
</font>

<font color="blue">

#### j) A intensidade da associação significante entre as duas variáveis, recorrendo ao odds ratio (OR), por o estudo ser transversal. Determine a discrepância entre OR e RP.
</font>

In [15]:
calcular_odds_ratio_razao_risco_discrepancia(tabela_contingencia, print_results=True)

,Value,95% CI Lower,95% CI Upper
Odds Ratio for Pobre (1. Favor de Pobre / 2. Contra Pobre),3.494,1.287,9.485
RR (ou RP) for Smãe = 1. Favor de Smãe,2.955,1.221,7.153
RR (ou RP) for Smãe = 2. Contra Smãe,0.846,0.739,0.967
N. of Valid Cases,156,,


Discrepância = 0.183 e (θ * p21) = 0.256


<font color="blue">
    
### 2.1.6. Probabiblidade de todos os acontecimentos possíveis
</font>

<font color="blue">

#### k) A probabilidade dos acontecimentos observados na Tabela 1, e de dois acontecimentos mais prováveis.
</font>

In [16]:
# Valores observados na amostra
observed_a11 = tabela_contingencia.iloc[0, 0]
observed_a12 = tabela_contingencia.iloc[0, 1]
observed_a21 = tabela_contingencia.iloc[1, 0]
observed_a22 = tabela_contingencia.iloc[1, 1]

# Definição dos totais marginais
M = tabela_contingencia.sum().sum() # total da amostra
n = observed_a11 + observed_a12 # total marginal da linha
N = observed_a11 + observed_a21 # total marginal da coluna

# Frequências esperadas
expected_a11 = frequencias_esperadas.iloc[0, 0]
# O inteiro mais próximo abaixo de expected_a11 é obtido pelo floor (arredondamento para baixo)
expected_a11_nearest_lower_integer = int(expected_a11)
# O inteiro mais próximo acima de expected_a11 é obtido pelo ceil (arredondamento para cima)
expected_a11_nearest_higher_integer = int(expected_a11) + 1

# Calculando a probabilidade do acontecimento observado
p_observed = hypergeom.pmf(observed_a11, M, n, N)

# Calculando as probabilidades dos dois acontecimentos mais prováveis
p_expected_lower = hypergeom.pmf(expected_a11_nearest_lower_integer, M, n, N)
p_expected_higher = hypergeom.pmf(expected_a11_nearest_higher_integer, M, n, N)

# Imprimindo
print(f"RESPOSTA:")
print(f"\nProbabilidade do acontecimento observado:")
print(f"P(a11={observed_a11}) = {p_observed:.4f}")
print(f"\nProbabilidades dois acontecimentos mais prováveis:")
print(f"P(a11={expected_a11_nearest_lower_integer}) = {p_expected_lower:.4f}")
print(f"P(a11={expected_a11_nearest_higher_integer}) = {p_expected_higher:.4f}")

RESPOSTA:

Probabilidade do acontecimento observado:
P(a11=16) = 0.0071

Probabilidades dois acontecimentos mais prováveis:
P(a11=10) = 0.1785
P(a11=11) = 0.1755


<font color="blue">

#### l)	Gerar probabilidades dos acontecimentos possíveis

</font>

In [17]:
df_formated, acceptance_range, rejection_range = calcular_distribuicao_probabilidades_e_decisao_hipotese(tabela_contingencia, 0.05)

print(f"RESULTADOS:\r")
display(df_formated.style.hide(axis='index'))

print(f"Região de Aceitação: {acceptance_range}")
print(f"Região de Rejeição: {rejection_range}")

RESULTADOS:


y_value,p_value,cum_sum
10,0.178,0.178
11,0.175,0.354
9,0.148,0.502
12,0.141,0.643
8,0.099,0.742
13,0.092,0.834
7,0.054,0.888
14,0.049,0.936
6,0.023,0.960
15,0.021,0.980


Região de Aceitação: [6, 14]
Região de Rejeição: [0, 5] U [15, 22]
